# 01. Import libraries and data

In [2]:
# Import libraries

import pandas as pd
import numpy as np
import os
# Set path
path = r'/home/scruffy/anaconda_projects/Instacart Basket Analysis/'
# Import data to dataframe
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_cust_added_cols.pkl'))

In [3]:
# Check import
print(ords_prods_merge.shape)
print(ords_prods_merge.head(10))

(32404859, 25)
   product_id                product_name  aisle_id  department_id    prices  \
0           1  Chocolate Sandwich Cookies        61             19  5.800781   
1           1  Chocolate Sandwich Cookies        61             19  5.800781   
2           1  Chocolate Sandwich Cookies        61             19  5.800781   
3           1  Chocolate Sandwich Cookies        61             19  5.800781   
4           1  Chocolate Sandwich Cookies        61             19  5.800781   
5           1  Chocolate Sandwich Cookies        61             19  5.800781   
6           1  Chocolate Sandwich Cookies        61             19  5.800781   
7           1  Chocolate Sandwich Cookies        61             19  5.800781   
8           1  Chocolate Sandwich Cookies        61             19  5.800781   
9           1  Chocolate Sandwich Cookies        61             19  5.800781   

   order_id  user_id  order_number  order_dow  order_hour_of_day  ...  gender  \
0   3139998      138   

# 02. Grouping and aggregating

In [5]:
# Print summary statistics by department for full data set
print(ords_prods_merge.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']}))

              order_number        
                      mean min max
department_id                     
1                15.457838   1  99
2                17.277920   1  99
3                17.170395   1  99
4                17.811403   1  99
5                15.215751   1  99
6                16.439806   1  99
7                17.225802   1  99
8                15.340650   1  99
9                15.895474   1  99
10               20.197148   1  99
11               16.170638   1  99
12               15.887671   1  99
13               16.583536   1  99
14               16.773669   1  99
15               16.165037   1  99
16               17.665606   1  99
17               15.694469   1  99
18               19.310397   1  99
19               17.177343   1  99
20               16.473447   1  99
21               22.902379   1  99


#### The results are very similar to the results for the same operation on the subsetted data, although in general the more extreme values are closer to the overall mean this time and the max values are all exactly 99.

### Create max_order and loyalty flag columns

In [8]:
# Create max_order column with the maximum order_number per user
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform('max')

In [9]:
# Create loyalty_flag column and define conditions for each tag
ords_prods_merge.loc[ords_prods_merge['max_order'] > 47, 'loyalty_flag'] = 'Loyal customer'

In [10]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 47) & (ords_prods_merge['max_order'] > 13), 'loyalty_flag'] = 'Regular customer'

In [11]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 13, 'loyalty_flag'] = 'New customer'

In [12]:
# Check results of loyalty_flag column
print(ords_prods_merge['loyalty_flag'].value_counts(dropna=False))

loyalty_flag
Regular customer    15941386
New customer         8494382
Loyal customer       7969091
Name: count, dtype: int64


In [13]:
# Print summary statistics for prices by each loyalty_flag group
prices_by_loyalty = ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max', 'median']})
print(prices_by_loyalty)

                    prices                     
                      mean  min   max    median
loyalty_flag                                   
Loyal customer    7.767535  1.0  25.0  7.398438
New customer      7.800277  1.0  25.0  7.398438
Regular customer  7.797513  1.0  25.0  7.398438


#### No observable difference in prices by loyalty bucket. These are per-item prices, however, so it is still possible that there could be differences in order size.

### Create avg_spend and avg_spend_flag columns

In [16]:
# Create avg_spend column with average of 'prices' by user
ords_prods_merge['avg_spend'] = ords_prods_merge.groupby(['user_id'])['prices'].transform('mean')

In [17]:
# Create avg_spend_flag column and define conditions for each tag
ords_prods_merge.loc[ords_prods_merge['avg_spend'] < 10, 'avg_spend_flag'] = 'Low spender'

In [18]:
ords_prods_merge.loc[ords_prods_merge['avg_spend'] >= 10, 'avg_spend_flag'] = 'High spender'

In [19]:
# Check results of avg_spend_flag column
print(ords_prods_merge['avg_spend_flag'].value_counts(dropna=False))

avg_spend_flag
Low spender     32285390
High spender      119469
Name: count, dtype: int64


### Create order_frequency and order_frequency_flag columns

In [21]:
# Create order_frequency columns using median of days_since_prior_order by user
ords_prods_merge['order_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform('median')

In [22]:
# Create order_frequency_flag column and define conditions for each tag
ords_prods_merge.loc[ords_prods_merge['order_frequency'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'

In [23]:
ords_prods_merge.loc[(ords_prods_merge['order_frequency'] > 10) & (ords_prods_merge['order_frequency'] <= 20), 'order_frequency_flag'] = 'Regular customer'

In [24]:
ords_prods_merge.loc[ords_prods_merge['order_frequency'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [25]:
# Check results of order_frequency_flag column
print(ords_prods_merge['order_frequency_flag'].value_counts(dropna=False))

order_frequency_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: count, dtype: int64


#### Results show 5 null values in order_frequency_flag column. It's possible that these are from users who've placed only one order and therefore the median of their days_since_prior_order is null. We can check this using the max_order column.

In [27]:
ords_prods_merge[ords_prods_merge['max_order']==1]

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_dow,order_hour_of_day,...,income,price_range,busiest_days_loc,busiest_period_of_day,max_order,loyalty_flag,avg_spend,avg_spend_flag,order_frequency,order_frequency_flag
6234909,10749,Organic Red Bell Pepper,83,4,5.699219,895835,159838,1,0,17,...,140994,Mid-range product,Busiest days,Average orders,1,New customer,7.419726,Low spender,NaN,NaN
12947653,21334,Organic Peeled Garlic,123,4,10.203125,895835,159838,1,0,17,...,140994,Mid-range product,Busiest days,Average orders,1,New customer,7.419726,Low spender,NaN,NaN
13839012,22198,4X Ultra Concentrated Natural Laundry Detergen...,75,17,1.700195,895835,159838,1,0,17,...,140994,Low-range product,Busiest days,Average orders,1,New customer,7.419726,Low spender,NaN,NaN
14758536,23695,California Veggie Burger,42,1,4.699219,895835,159838,1,0,17,...,140994,Low-range product,Busiest days,Average orders,1,New customer,7.419726,Low spender,NaN,NaN
21673807,33401,Goat Cheese Crumbles,21,16,14.796875,895835,159838,1,0,17,...,140994,Mid-range product,Busiest days,Average orders,1,New customer,7.419726,Low spender,NaN,NaN


#### As suspected, we have five observations from users with a max_order of 1. This is an infrequent user and so can be added to that bucket.

In [29]:
# Mark users with max_order of 1 as infrequent customers
ords_prods_merge.loc[(ords_prods_merge['max_order']==1), 'order_frequency_flag'] = 'Non-frequent customer'

In [30]:
# Check updated results
print(ords_prods_merge['order_frequency_flag'].value_counts(dropna=False))

order_frequency_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636442
Name: count, dtype: int64


# 03. Check and export data

In [32]:
# Check updated DataFrame
print(ords_prods_merge.head())
print(ords_prods_merge.info())
print(ords_prods_merge.describe())

   product_id                product_name  aisle_id  department_id    prices  \
0           1  Chocolate Sandwich Cookies        61             19  5.800781   
1           1  Chocolate Sandwich Cookies        61             19  5.800781   
2           1  Chocolate Sandwich Cookies        61             19  5.800781   
3           1  Chocolate Sandwich Cookies        61             19  5.800781   
4           1  Chocolate Sandwich Cookies        61             19  5.800781   

   order_id  user_id  order_number  order_dow  order_hour_of_day  ...  income  \
0   3139998      138            28          6                 11  ...   49620   
1   1977647      138            30          6                 17  ...   49620   
2    389851      709             2          0                 21  ...  158302   
3    652770      764             1          3                 13  ...   31308   
4   1813452      764             3          4                 17  ...   31308   

         price_range  busiest_da

/home/scruffy/anaconda3/lib/python3.12/site-packages/pandas/core/nanops.py:1487: RuntimeWarning: overflow encountered in cast
  return dtype.type(n)
/home/scruffy/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/scruffy/anaconda3/lib/python3.12/site-packages/pandas/core/nanops.py:731: RuntimeWarning: invalid value encountered in scalar divide
  the_mean = the_sum / count if count > 0 else np.nan
/home/scruffy/anaconda3/lib/python3.12/site-packages/pandas/core/nanops.py:1487: RuntimeWarning: overflow encountered in cast
  return dtype.type(n)
/home/scruffy/anaconda3/lib/python3.12/site-packages/pandas/core/nanops.py:1487: RuntimeWarning: overflow encountered in cast
  return dtype.type(n)
/home/scruffy/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keep

         product_id      aisle_id  department_id        prices      order_id  \
count  3.240486e+07  3.240486e+07   3.240486e+07  3.239973e+07  3.240486e+07   
mean   2.559866e+04  6.449111e+01   9.919792e+00           NaN  1.710745e+06   
std    1.408400e+04  4.843815e+01   6.281485e+00  0.000000e+00  9.872988e+05   
min    1.000000e+00 -1.280000e+02   1.000000e+00  1.000000e+00  2.000000e+00   
25%    1.354400e+04  2.400000e+01   4.000000e+00  4.199219e+00  8.559470e+05   
50%    2.530200e+04  7.900000e+01   9.000000e+00  7.398438e+00  1.711049e+06   
75%    3.794700e+04  1.000000e+02   1.600000e+01  1.129688e+01  2.565499e+06   
max    4.968800e+04  1.270000e+02   2.100000e+01  2.500000e+01  3.421083e+06   

            user_id  order_number     order_dow  order_hour_of_day  \
count  3.240486e+07  3.240486e+07  3.240486e+07       3.240486e+07   
mean   1.029372e+05  1.714230e+01  2.738867e+00       1.342515e+01   
std    5.946610e+04  1.753532e+01  2.090077e+00       4.246380e+00   

In [33]:
# Export data to pkl
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_cust_new_cols.pkl'))